In [1]:
!pip install datasets -q

In [ ]:
import os

In [2]:
import json
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

## Data loading

In [3]:
!wget https://github.com/anna-a-m/dataset-2rca/raw/refs/heads/main/data/2rca_checked_version.json -q

In [4]:
with open('2rca_checked_version.json') as f:
    data = json.load(f)

## Data samples

In [5]:
data[0]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_64c12a9b',
 'Phrase': 'Что читаешь? Мне нравится классика. Я тоже люблю пообщаться.',
 'Rewrite': 'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'}

In [6]:
data[1]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_5cd9dc06',
 'Phrase': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику люблю.',
 'Rewrite': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.'}

In [7]:
data[2]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_9d73f6c2',
 'Phrase': 'Ух ты, интересно.',
 'Rewrite': 'Ух ты, интересно, ты фиалки выращиваешь.'}

In [8]:
data[3]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.',
  'Ух ты, интересно, ты фиалки выращиваешь.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_2bfa24e4',
 'Phrase': 'Ты случайно не принц на белом коне? Я его очень жду.',
 'Rewrite': 'Ты случайно не принц на белом коне? Я принца на белом коне очень жду.'}

Saving sample of data for manual examination

In [9]:
history, phrase, rewrite = [], [], []

for sample in data:
    history.append(sample["History"])
    phrase.append(sample["Phrase"])
    rewrite.append(sample["Rewrite"])

In [10]:
df = pd.DataFrame({"history": history, "phrase": phrase, "rewrite": rewrite})

## HF datasets

In [11]:
from datasets import DatasetDict, Dataset

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_ds, test_ds = ds["train"], ds["test"]
test_ds = test_ds.train_test_split(test_size=0.5, shuffle=True, seed=42)
val_ds, test_ds = test_ds["train"], test_ds["train"]

ds["train"] = train_ds
ds["val"] = val_ds
ds["test"] = test_ds

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
})

In [13]:
!pip install together -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
from together import Together

client = Together()

In [48]:
from tqdm import tqdm

In [49]:
prompt = "Ты професиональный разметчик с лингвистическим образованием. Внимательно прочитай диалог. Перепишите неполное высказывание на основе истории диалога. Ответ должен содержать только переписанное высказывание\n"

In [50]:
MODEL_NAME = "deepseek-ai/DeepSeek-R1"

In [51]:
ds['train'][0]

{'history': ['Привет.',
  'Привет.',
  'Как твоё настроение сегодня?',
  'Настроение отлично, скоро куплю машину, а твоё настроение как?',
  'Здорово, ты скоро купишь машину! У меня тоже настроение на позитиве! Я очень давно хочу в Бразилию, и, кажется, сегодня моя мечта сбудется! Вечером поедем с подругой в турфирму оплачивать наш тур в Бразилию. А у тебя есть мечта?',
  'Давно хочу завести лабрадора, прям с детства хочу собаку. Может, когда дочка подрастет заведу себе лабрадора.',
  'Моей собаке уже 5 лет, и я даже не представляю, как я могла жить без своей собаки раньше?! Я думаю, что у тебя всё получится и у вас скоро обязательно появится питомец! Ведь собаки такие милые! Что сегодня будешь готовить на ужин?'],
 'phrase': 'Сегодня будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.',
 'rewrite': 'Сегодня на ужин будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'}

In [53]:
import time
raw_test_results_new = []


for i in tqdm(range(100)):
    success = False
    while not success:
        try:
            model=MODEL_NAME,
            messages = [
                {
                    "role": "system",
                    "content": prompt
                },

                {
                    "role": "user",
                    "content": "История: " + str(ds['train']['history'][0][-1]) + " Неполное высказвание: " + ds['train']["phrase"][0]
                },
                {
                    "role": "assistant",
                    "content": ds['train']['rewrite'][0]
                },

                {
                    "role": "user",
                    "content": "История: " + str(ds['train']['history'][1][-1]) + " Неполное высказвание: " + ds['train']["phrase"][1]
                },
                {
                    "role": "assistant",
                    "content": ds['train']['rewrite'][1]
                },

                {
                    "role": "user",
                    "content": "История: " + str(ds['train']['history'][2][-1]) + " Неполное высказвание: " + ds['train']["phrase"][2]
                },
                {
                    "role": "assistant",
                    "content": ds['train']['rewrite'][2]
                },


                {
                    "role": "user",
                    "content": "История: " + str(ds['test']['history'][i][-1]) + " Неполное высказвание: " + ds['test']["phrase"][i]
                    }
                ]

            response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=1
        )
            out = response.choices[0].message.content
            raw_test_results_new.append(out)
            success = True
        except:
            time.sleep(20)


 52%|█████▏    | 52/100 [3:00:36<2:46:43, 208.40s/it]


KeyboardInterrupt: 

In [54]:
len(raw_test_results_new)

52

In [55]:
print(raw_test_results_new[0])

<think>
Okay, let's see here. The user is acting like a professional annotator with a linguistics background. They've given me a history and an incomplete statement to rewrite based on the dialog history. The task is to only provide the rewritten statement.

Looking at the history: "Привет. Откуда ты?" (Hello. Where are you from?), and the incomplete response is "Я из Твери, а ты?" (I'm from Tver, and you?).

The previous examples show that the assistant expanded the incomplete statements by making them more complete, while maintaining the original meaning. For instance, "Дима, а тебя?" became "Меня зовут Дима, а тебя как зовут?" which is a more complete question.

So in this case, the incomplete response "Я из Твери, а ты?" needs to be expanded. The user probably expects a more structured sentence, like adding the verb or question structure. In Russian, a common way to respond would be "Я из Твери, а ты откуда?" which specifies the question is about origin. Alternatively, it could be 

In [ ]:
raw_test_results_old = raw_test_results_new

In [58]:
raw_test_results_new = [r.split("</think>")[-1] for r in raw_test_results_new]

In [72]:
import pandas as pd

raw_test_results = pd.DataFrame(raw_test_results_new, columns=['model_out_raw'])

raw_test_results['history'] = ds['test']['history'][:52]
raw_test_results['text'] = ds['test']['phrase'][:52]
raw_test_results['restored_text'] = ds['test']['rewrite'][:52]

raw_test_results.head(10)

,model_out_raw,history,text,restored_text
0,"\n\nЯ из Твери, а ты откуда?","[Привет., Привет. Откуда ты?]","Я из Твери, а ты?","Я из Твери, а ты откуда?"
1,"\n\nДа, именно Питерский политех. Жаль, что уч...","[Привет., Привет! Сколько тебе лет? Ты где-то ...","Да, именно Питерский. Жаль, что не в месте учи...","Да, именно Питерский политех. Жаль, что не вме..."
2,\n\nМои дети — это мои песики. Пока не встрети...,"[Приветики! Как дела?, Привет! Всё хорошо, а у...",Мои дети - мои песики. Пока не нашла достойног...,Мои дети - мои песики. Пока не нашла достойног...
3,\n\nА где ты живёшь? В городе? И какие фрукты ...,"[Привет., Привет., Расскажи о себе., Я собираю...",Где живёшь? В городе где-то? Какие любишь?,Где живёшь? В городе где-то? Какие фрукты любишь?
4,"\n\nДа, это самое главное. А с кем ты работаешь?","[Здравствуй. Как тебе погодка?, Привет, я дума...","Да, это самое главное, с кем ты работаешь?","Да, это самое главное что весна придет через п..."
5,"\n\nИногда беру ее (собаку) с собой, а у тебя ...","[Привет, ты откуда? Я фотограф. Люблю путешест...","Иногда беру ее с собой, у тебя кто?","Иногда беру дочь с собой в путешествие, а у те..."
6,\n\nЯ всегда за новые знакомства.,"[Добрый вечер, вы не против новых знакомств?]",Я всегда за.,Я всегда за знакомства.
7,\n\nЯ певица. А вы кем работаете?,"[Привет., Привет, Алла., Чем вы занимаетесь?]",Певица. А вы?,Я певица. А вы чем занимаетесь?
8,\n\nЧтобы не испортить зрение и осанку из-за д...,"[Привет., О, привет! Рад новому знакомству! Ты...",Чтобы не испортить зрение и осанку за компьюте...,Чтобы не испортить зрение и осанку за компьюте...
9,"\n\nДа, я и учусь, и работаю.","[Привет., Привет. Как дела?, Отлично у меня де...","Да, и учусь, и работаю.","Да, я и учусь, и работаю."


In [77]:
raw_test_results = raw_test_results[~raw_test_results.model_out_raw.map(lambda x: x.startswith("<think>"))]

In [78]:
raw_test_results.to_csv(f"{MODEL_NAME.split('/')[-1]}_raw_test_results.csv")

In [79]:
!pip install evaluate rouge_metric -q
!pip install sacrebleu -q

In [80]:
import numpy as np
import evaluate
from rouge_metric import PyRouge

bleu = evaluate.load("sacrebleu")
rouge = PyRouge(rouge_n=(4), skip_gap=4)


class RestorationFScore:

    def __init__(self, n_gram: int=2):
        self.n_gram = n_gram
        self.tokenizer = tokenizer

    def preprocess(self, sents):
        for sent in sents:
            sent_tokenize = self.tokenizer.encode(sent)
            yield [tuple(sent_tokenize[i:i+self.n_gram]) for i, _ in enumerate(sent_tokenize)]

    def _itereval(self):
        for i, predictions in enumerate(self.predictions):
            restored_ngrams = set(predictions).difference(self.references[i])
            ngrams_in_ref = set(self.rewrites[i]).difference(self.references[i])
            interagree = ngrams_in_ref.intersection(restored_ngrams)
            if len(restored_ngrams):
                precision = len(interagree) / len(restored_ngrams)
            else:
                precision = 0.
            if len(ngrams_in_ref):
                recall = len(interagree) / len(ngrams_in_ref)
            else:
                recall = 0.
            if precision or recall:
                yield 2 * ((precision * recall) / (precision + recall))
            else:
                yield 0.

    def evaluate(self, predictions: list,
                 references: list, rewrites: list):
        self.predictions = [p for p in self.preprocess(predictions)]
        self.references = [p for p in self.preprocess(references)]
        self.rewrites = [p for p in self.preprocess(rewrites)]
        return np.mean(list(self._itereval()))

In [81]:
def callculate_metrics(row):
    row["bleu_score"] = bleu.compute(predictions=row.model_out_raw,
                                     references=row.text)['score']

    rouge_scores = rouge.evaluate(row.model_out_raw,
                                  [[t] for t in row.text])
    for k in rouge_scores:
        row[k] = rouge_scores[k]['f']

    for n in range(1, 5):
        rf_score = RestorationFScore(n)
        row[f"rf_score_{n}"] = rf_score.evaluate(predictions=row.model_out_raw,
                                                 references=row.text,
                                                 rewrites=row.restored_text)
    return row

In [82]:
raw_test_results["type"] = "2rca"
raw_test_results = raw_test_results.groupby(by="type").agg(list)

<ipython-input-82-2a0a5f4aee2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_test_results["type"] = "2rca"


In [83]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1")

In [84]:
raw_test_results.apply(callculate_metrics, axis=1).drop(columns=["model_out_raw", "history", "text", "restored_text"])

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,41.75599,0.560716,0.373565,0.254357,0.164917,0.542418,0.35734,0.272433,0.241606,0.21401
